In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf

tf.get_logger().setLevel("ERROR")
import tensorflow_datasets as tfds

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp


# Using Bicubic Downscaling


In [ ]:
scale_factor = 4
resize_fn = dppp.create_resize_fn("bicubic", True)

lf_dataset = tfds.load("hci_lf", split="test").map(
    datasets.compose(
        datasets.map_on_dict_key("lf", datasets.to_float32),
        datasets.map_on_dict_key("lf", datasets.from_255_to_1_range),
    )
)

example = datasets.get_one_example(lf_dataset, index=0)
lf_hr = example["lf"]
lf_lr = dppp.resize_lf(lf_hr, scale_factor, False, resize_fn=resize_fn)


### Reconstruct


In [ ]:
# Load the denoiser
denoiser, (denoiser_min, denoiser_max) = dppp.load_denoiser("models/drugan+_0.0-0.2.h5")

# Callbacks: Print the PSNR every 5th step
callbacks = [dppp.callback_print_psnr("psnr", 5, lf_hr[4, 4])]

reconstructed = dppp.hqs_lf_super_resolve(
    degraded=lf_lr,
    sr_factor=scale_factor,
    denoiser=denoiser,
    max_denoiser_stddev=denoiser_max,
    resize_fn=resize_fn,
    callbacks=callbacks,
)


In [ ]:
# Print PSNR and LPIPS
psnr = dppp.psnr(lf_hr[4, 4], reconstructed[0]).numpy()
lpips = dppp.lpips_alex(lf_hr[4, 4], reconstructed[0]).numpy()
print(f"Reconstructed PSNR: {psnr:0.2f}, LPIPS: {lpips:0.4f}")

# Visualize
visualize.draw_images([lf_hr[4, 4], lf_lr[4, 4], reconstructed[0]])


# Using a Gaussian Blur Kernel


In [ ]:
scale_factor = 4
example = datasets.get_one_example(lf_dataset, index=1)
lf_hr = example["lf"]

kernel = dppp.conv2D_filter_rgb(dppp.ZHANG_GAUSSIAN_KERNELS[0])
lf_lr = dppp.resize_lf(
    lf_hr, scale_factor, False, dppp.create_convolve_resize_fn(kernel, mode="wrap")
)


### Reconstruct


In [ ]:
# Load the denoiser
denoiser, (denoiser_min, denoiser_max) = dppp.load_denoiser("models/drugan+_0.0-0.2.h5")

# Callbacks: Print the PSNR every 5th step
callbacks = [dppp.callback_print_psnr("psnr", 5, lf_hr[4, 4])]

reconstructed = dppp.hqs_lf_super_resolve(
    degraded=lf_lr,
    sr_factor=scale_factor,
    denoiser=denoiser,
    max_denoiser_stddev=denoiser_max,
    kernel=kernel,
    callbacks=callbacks,
)


In [ ]:
# Print PSNR and LPIPS
psnr = dppp.psnr(lf_hr[4, 4], reconstructed[0]).numpy()
lpips = dppp.lpips_alex(lf_hr[4, 4], reconstructed[0]).numpy()
print(f"Reconstructed PSNR: {psnr:0.2f}, LPIPS: {lpips:0.4f}")

# Visualize
visualize.draw_images([lf_hr[4, 4], lf_lr[4, 4], reconstructed[0]])
